In [1]:
import json

# Read the omni_math.json file from the dataset folder
with open('dataset/omni_math.json', 'r') as file:
    omni_math_data = json.load(file)

In [2]:
from unsloth import FastLanguageModel
from transformers import TrainingArguments
from datasets import load_dataset

# ✅ Load model (choose one)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-4-mini-instruct-unsloth-bnb-4bit", # or "microsoft/phi-3-mini-4k-instruct"
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

# ✅ Load your dataset (Alpaca style)
dataset = load_dataset("json", data_files="omni_math.json", split="train")

# ✅ Format to SFT
FastLanguageModel.prepare_model_for_training(model)

# ✅ Train
training_args = TrainingArguments(
    output_dir = "outputs/phi4mini-math-unsloth",
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4,
    num_train_epochs = 3,
    learning_rate = 2e-4,
    bf16 = False,
    fp16 = True,
    logging_steps = 10,
    save_steps = 200,
    save_total_limit = 2,
)

model.train()
model = FastLanguageModel.get_peft_model(
    model,
    r = 8,
    lora_alpha = 16,
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM",
)

model.fit(
    dataset=dataset,
    tokenizer=tokenizer,
    args=training_args,
)

NotImplementedError: Unsloth currently only works on NVIDIA GPUs and Intel GPUs.

In [1]:
import torch

torch.__version__

'2.7.0+cpu'